In [7]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [2]:
# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.

input_texts = [
    # Animal-related queries
    "query: Lions are known as the kings of the jungle.",
    "query: Dolphins are highly intelligent marine animals.",
    "query: Birds migrate to warmer climates during winter.",
    
    # Real estate-related queries
    "query: A well-maintained property can increase in value over time.",
    "query: Renting is often more flexible than buying a home.",
    "query: Many cities are experiencing a housing shortage.",
    
    # Computer-related queries
    "query: Machine learning is a subset of artificial intelligence.",
    "query: Python is a popular programming language for beginners.",
    "query: Cybersecurity is essential in today's digital world.",
    
    # Food-related queries
    "query: Italian pizza is famous worldwide for its rich flavors.",
    "query: Eating fresh fruits daily can improve your health.",
    "query: Spices like turmeric and cumin are staples in Indian cuisine."
]

In [3]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

In [4]:
# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

In [5]:
outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

In [11]:
# Scegli il numero di cluster (puoi modificarlo o calcolarlo con metodi come silhouette score)
num_clusters = 4

# Applichiamo k-means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)

embeddings_np = embeddings.detach()
kmeans.fit(embeddings_np)

# Otteniamo i gruppi
labels = kmeans.labels_

# Raggruppiamo gli indici degli embedding in base ai cluster
groups = {i: [] for i in range(num_clusters)}
for idx, label in enumerate(labels):
    groups[label].append(idx)

# Stampiamo i gruppi
print("Raggruppamento degli embedding in base ai cluster:")
for cluster, indices in groups.items():
    print(f"Cluster {cluster}: {indices}")

Raggruppamento degli embedding in base ai cluster:
Cluster 0: [3, 4, 5]
Cluster 1: [6, 7, 8]
Cluster 2: [0, 1, 2]
Cluster 3: [9, 10, 11]


c:\Users\Riccardo\.conda\envs\data_science_complete\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [12]:
# Step 1: Vectorize the text
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(input_texts)

# Step 2: Cluster the text using KMeans
n_clusters = 4  # Number of expected groups
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
labels = kmeans.fit_predict(X)

# Step 3: Generate topic titles by analyzing cluster centers
terms = np.array(vectorizer.get_feature_names_out())
topic_titles = []

for i in range(n_clusters):
    cluster_center = kmeans.cluster_centers_[i]
    top_terms = terms[np.argsort(-cluster_center)[:3]]  # Top 3 terms per cluster
    topic_titles.append("Topic: " + ", ".join(top_terms))

# Assign titles to each group
grouped_texts = {topic_titles[i]: [] for i in range(n_clusters)}

for i, label in enumerate(labels):
    grouped_texts[topic_titles[label]].append(input_texts[i])

# Display the results
import pprint
pprint.pprint(grouped_texts)

{'Topic: artificial, intelligence, learning': ['query: Machine learning is a '
                                               'subset of artificial '
                                               'intelligence.'],
 'Topic: cumin, spices, staples': ['query: Spices like turmeric and cumin are '
                                   'staples in Indian cuisine.'],
 'Topic: query, kings, jungle': ['query: Lions are known as the kings of the '
                                 'jungle.',
                                 'query: Dolphins are highly intelligent '
                                 'marine animals.',
                                 'query: Birds migrate to warmer climates '
                                 'during winter.',
                                 'query: A well-maintained property can '
                                 'increase in value over time.',
                                 'query: Renting is often more flexible than '
                                 'buying a h

In [13]:
# https://github.com/princeton-nlp/SimCSE/blob/main/README.md

In [14]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

c:\Users\Riccardo\.conda\envs\data_science_complete\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

c:\Users\Riccardo\.conda\envs\data_science_complete\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [16]:
ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

In [17]:
s = summarizer(ARTICLE, max_length=150, min_length=50, do_sample=False)

In [18]:
s[0]['summary_text']

'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'